# Backtest the strategies

Use an LLM to go through and predict the buy/ sell/ hold recommendation for the company for the given date. Steps needed:

1. Load the LLM - use DeepSeek R1 Qwen model at 7B parameters first and try the quantised models next
2. Step through each data and each financial statement to get a result
3. Log the results in a file and save to S3 (will need a logging file to save to S3 and resume in case of kernel crash)
4. Need a backtesting framework to apply the results


## Load libraries needed

In [116]:
import json
import boto3
from s3fs import S3FileSystem
import os
import datetime

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch
from accelerate import Accelerator

import pandas as pd
from IPython.display import Markdown, display
from ipywidgets import IntProgress, Label, HBox

from helper import get_s3_folder
import s3Helpers
import company_data
from s3Helpers import S3ModelHelper, Logger
from prompts import SYSTEM_PROMPTS

In [48]:
import importlib
importlib.reload(company_data)
importlib.reload(s3Helpers)

<module 's3Helpers' from '/project/s3Helpers.py'>

## Load the LLM

Models to test:
- Qwen (Qwen/Qwen2.5-7B-Instruct)
- Llama (meta-llama/Llama-3.2-7B-Instruct)
- DeepSeek (deepseek-ai/DeepSeek-R1-Distill-Qwen-14B)

In [3]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [4]:
# Set up Quantization 
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"

)

In [53]:
accelerator = Accelerator()

In [5]:
# Flag to download from Huggingface again or use stored model
USE_HF = False
USE_QUANTIZATION = False

model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id_s3 = 'llama'


if USE_HF:
   
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    
    if USE_QUANTIZATION:
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', quantization_config=quant_config)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
    model_helper = s3Helpers.S3ModelHelper(s3_sub_folder='tmp/fs')
    if USE_QUANTIZATION:
        model = model_helper.load_model(model_id_s3, quant_config)
    else:
        model = model_helper.load_model(model_id_s3)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    model_helper.clear_folder(model_id_s3)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


## Load Financial PIT dataset

In [84]:
## Load from S3 using the helper file
filename = 'data_quarterly_pit_indu.json'
sec_helper = company_data.SecurityData('tmp/fs',filename)
all_data = sec_helper.get_all_data()

In [86]:
# USE WHILE DEVELOPING to
importlib.reload(company_data)
sec_helper = company_data.SecurityData('tmp/fs',filename, all_data)

In [87]:
sec_helper.total_securities_in_backtest()

896

In [71]:
sec_helper.get_security_statement('2020-04-24','AXP UN Equity','px') #AXP UN Equity2020-04-24

,Price
Date,
2019-04-24,114.02
2019-05-24,119.51
2019-06-24,124.14
2019-07-24,127.95
2019-08-24,117.76
2019-09-24,118.17
2019-10-24,116.41
2019-11-24,119.06
2019-12-24,124.74


In [118]:
system_prompt = SYSTEM_PROMPTS['BASE']['prompt']
system_prompt

"You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets.         Compute common financial ratios and then determine the buy or sell decision. Explain your reasons in less than 250 words. Provide a         confidence score for how confident you are of the decision. If you are not confident then lower the confidence score.         Your answer must be in a JSON object. Provide your answer in JSON format like the two examples below:         {'decision': BUY, 'confidence score': 80, 'reason': 'Gross profit and EPS have both increased over time'}         {'decision': SELL, 'confidence score': 90, 'reason': 'Price has declined and EPS is falling'}"

In [72]:
prompt = sec_helper.get_prompt('2020-04-24','AXP UN Equity', system_prompt)

In [73]:
tokens = tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True)
len(tokens)

2895

## Run an example in LLM

Run into out of memory problem - Potential fixes:
1. reduce size of model (quantize)
2. explore multi-gpu
3. reduce tokens

https://saturncloud.io/blog/how-to-solve-cuda-out-of-memory-error-in-pytorch/

https://huggingface.co/docs/accelerate/usage_guides/distributed_inference

https://medium.com/@geronimo7/llms-multi-gpu-inference-with-accelerate-5a8333e4c5db

Problem with splitting a single prompt into multiple gpus to calculate the result. Tensor parallelism - https://huggingface.co/docs/transformers/main/en/perf_train_gpu_many#tensor-parallelism

nvidia-smi will show available GPUs on the system.

### Run 1
Run in 5.5 hours on 1 A10G GPU on 896 security/ date combinations. The data is stored in log files in the project for further analysis. 

In [74]:
def run_model(prompt, tokenizer, model):
    tokens = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([tokens], return_tensors='pt').to('cuda')
    generated_ids = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1000)
    parsed_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    return tokenizer.batch_decode(parsed_ids, skip_special_tokens=True)[0]



In [75]:
# Time the execution
start_time = datetime.datetime.now()

# Run the model
response = run_model(prompt, tokenizer, model)

#Print the length of time to run
end_time = datetime.datetime.now()
print("Time to execute: ", end_time - start_time)

Time to execute:  0:00:21.858961


In [76]:
display(Markdown(response))

Here is the JSON object with the financial ratios and buy/sell/hold decision:

```json
{
  "Decision": "HOLD",
  "confidence score": 60,
  "Reason": "Operating Income and Net Income have been declining over the past two years, but the company has shown a slight increase in revenue. The debt-to-equity ratio is also increasing, which may be a concern."
}
```

To calculate the financial ratios, I used the following:

1. Debt-to-equity ratio: Total Liabilities / Total Equity
2. Return on Equity (ROE): Net Income / Total Equity
3. Current Ratio: Current Assets / Current Liabilities
4. Interest Coverage Ratio: Earnings Before Interest and Taxes (EBIT) / Interest Expense

Here are the calculations:

1. Debt-to-equity ratio:
   - 2020: 1.663120e+11 / 2.100600e+10 = 0.794
   - 2019: 1.752500e+11 / 2.307100e+10 = 0.762
   - 2018: 1.711590e+11 / 2.302500e+10 = 0.743

The debt-to-equity ratio has been increasing over the past three years, which may indicate that the company is taking on more debt to fund its operations.

2. Return on Equity (ROE):
   - 2020: 2.229000e+10 / 2.100600e+10 = 1.061
   - 2019: 2.307100e+10 / 2.307100e+10 = 1.000
   - 2018: 2.302500e+10 / 2.302500e+10 = 1.000

The ROE has been declining over the past three years, which may indicate that the company is not generating enough profits from its operations.

3. Current Ratio:
   - 2020: 1.886020e+11 / 1.650540e+11 = 1.140
   - 2019: 1.983210e+11 / 1.752500e+11 = 1.120
   - 2018: 1.941840e+11 / 1.711590e+11 = 1.130

The current ratio has been declining over the past three years, which may indicate that the company is not generating enough cash from its operations to cover its liabilities.

4. Interest Coverage Ratio:
   - 2020: 2.229000e+10 / 3.046000e+09 = 7.373
   - 2019: 2.307100e+10 / 3.085000e+09 = 7.479
   - 2018: 2.302500e+10 / 3.080000e+09 = 7.473

The interest coverage ratio has been declining over the past three years, which may indicate that the company is not generating enough cash from its operations to cover its interest expenses.

Based on these ratios, I am recommending a HOLD decision due to the declining operating income, increasing debt-to-equity ratio, declining current ratio, and declining interest coverage ratio. However, the company has shown a slight increase in revenue over the past year, which may indicate some potential for growth.

In [80]:
def format_json(llm_output):
    form = llm_output.replace('\n','')
    soj = form.find('```json')
    eoj = form.find('}```')
    additional = form[eoj + 4:]
    json_obj = json.loads(form[soj + 7:eoj + 1])
    json_obj['AdditionalContext'] = additional
    return json_obj

In [78]:
response

'Here is the JSON object with the financial ratios and buy/sell/hold decision:\n\n```json\n{\n  "Decision": "HOLD",\n  "confidence score": 60,\n  "Reason": "Operating Income and Net Income have been declining over the past two years, but the company has shown a slight increase in revenue. The debt-to-equity ratio is also increasing, which may be a concern."\n}\n```\n\nTo calculate the financial ratios, I used the following:\n\n1. Debt-to-equity ratio: Total Liabilities / Total Equity\n2. Return on Equity (ROE): Net Income / Total Equity\n3. Current Ratio: Current Assets / Current Liabilities\n4. Interest Coverage Ratio: Earnings Before Interest and Taxes (EBIT) / Interest Expense\n\nHere are the calculations:\n\n1. Debt-to-equity ratio:\n   - 2020: 1.663120e+11 / 2.100600e+10 = 0.794\n   - 2019: 1.752500e+11 / 2.307100e+10 = 0.762\n   - 2018: 1.711590e+11 / 2.302500e+10 = 0.743\n\nThe debt-to-equity ratio has been increasing over the past three years, which may indicate that the compan

In [81]:
format_json(response)

{'Decision': 'HOLD',
 'confidence score': 60,
 'Reason': 'Operating Income and Net Income have been declining over the past two years, but the company has shown a slight increase in revenue. The debt-to-equity ratio is also increasing, which may be a concern.',
 'AdditionalContext': 'To calculate the financial ratios, I used the following:1. Debt-to-equity ratio: Total Liabilities / Total Equity2. Return on Equity (ROE): Net Income / Total Equity3. Current Ratio: Current Assets / Current Liabilities4. Interest Coverage Ratio: Earnings Before Interest and Taxes (EBIT) / Interest ExpenseHere are the calculations:1. Debt-to-equity ratio:   - 2020: 1.663120e+11 / 2.100600e+10 = 0.794   - 2019: 1.752500e+11 / 2.307100e+10 = 0.762   - 2018: 1.711590e+11 / 2.302500e+10 = 0.743The debt-to-equity ratio has been increasing over the past three years, which may indicate that the company is taking on more debt to fund its operations.2. Return on Equity (ROE):   - 2020: 2.229000e+10 / 2.100600e+10 =

## Run the backtest and generate all responses

In [37]:
dates = sec_helper.get_dates()

In [93]:
importlib.reload(s3Helpers)
importlib.reload(company_data)

<module 'company_data' from '/project/company_data.py'>

In [113]:
logger = s3Helpers.Logger('tmp/fs')
def run_backtest(company_info, tokenizer, model, logger, log_at=50, start_count=0):
    # get the dates
    dates = company_info.get_dates()
    # set the current date year
    current_year = dates[0][:4]

    # set the array
    year_log = []
    
    # set up the display
    max_count = company_info.total_securities_in_backtest()
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    l = Label(value=str(f.value))
    display(HBox([f,l]))
    
    count = 0

    # run the backtest 
    for date in dates:
        
        securities = company_info.get_securities_reporting_on_date(date)

        for security in securities:
            if date[:4] != current_year or count % log_at == 0:
                # save the file to S3 and reset when it is a new year
                logger.log(year_log, current_year + str(count) + '.json')
                # reset the stats
                year_log = []
                current_year = date[:4]

        
            prompt = company_info.get_prompt(date, security, system_prompt)
            response = run_model(prompt, tokenizer, model)
            try:
                formatted_response = format_json(response)
                formatted_response['security'] = security
                formatted_response['date'] = date
                year_log.append(formatted_response)
            except:
                print("error with " + security + date)
                error_json = {'security': security, 'date': date, 'response': response}
                year_log.append(error_json)
            f.value += 1
            count += 1
            l.value = str(count) + "/" + str(max_count)
    # Log the last values
    logger.log(year_log, current_year + str(count) + '.json')

In [114]:
run_backtest(sec_helper, tokenizer, model, logger)

Saved 20200.json
error with JNJ UN Equity2020-02-18
error with CAT UN Equity2020-02-19
error with JPM UN Equity2020-04-14
error with CAT UN Equity2020-04-28
error with MRK UN Equity2020-04-28
error with AAPL UQ Equity2020-04-30
error with HON UW Equity2020-05-01
error with GS UN Equity2020-05-01
error with AMZN UQ Equity2020-05-01
error with HON UQ Equity2020-05-01
Saved 202050.json
error with XOM UN Equity2020-05-06
error with CSCO UW Equity2020-05-13
error with WMT UN Equity2020-05-19
error with WBA UW Equity2020-07-09
error with JPM UN Equity2020-07-14
error with JNJ UN Equity2020-07-16
error with KO UN Equity2020-07-22
error with DOW UN Equity2020-07-23
error with INTC UW Equity2020-07-24
error with HON UN Equity2020-07-24
error with AXP UN Equity2020-07-24
error with HON UW Equity2020-07-24
error with VZ UN Equity2020-07-28
error with MMM UN Equity2020-07-28
error with MCD UN Equity2020-07-28
error with IBM UN Equity2020-07-28
error with BA UN Equity2020-07-29
error with XOM UN Eq

### Concatenate all of the results

In [125]:
log_list = logger.get_list_of_logs()

In [129]:
log_list[0][log_list[0].find('/logs/') + 6:]

'20200.json'

In [133]:
def concat_all_logs():
    log_list = logger.get_list_of_logs()
    logs = []
    for logfile in log_list:
        logs += logger.get_log(logfile[logfile.find('/logs/') + 6:])
    return logs

In [134]:
logs = concat_all_logs()

In [135]:
len(logs)

879

In [57]:
test_json = [{'test':'test'}]

In [58]:
logger.log(test_json, 'test.json')

In [95]:
logger.get_list_of_logs()

['bclarke16/tmp/fs/logs/20200.json',
 'bclarke16/tmp/fs/logs/202010.json',
 'bclarke16/tmp/fs/logs/test.json']

In [97]:
 d = logger.get_log('202010.json')

In [104]:
len(d)

10

In [28]:
df = sec_helper.get_security_statement('2020-01-31','AON UN Equity','is')

In [ ]:
#with open('Data/base1.json', 'w') as file:
#    json.dump(logs, file)

In [ ]:
# start_time = datetime.datetime.now()
# #formatted_chat = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
# outputs = pipeline(
#     prompt,
#     max_new_tokens=1000,
# )
# end_time = datetime.datetime.now()
# print("Time to execute: ", end_time - start_time)

# test_output = outputs[0]['generated_text'][-1]
# display(Markdown(test_output['content']))